# Loading packages

In [1]:
import networkx as nx
import osmnx as ox
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import network_func
from matplotlib.collections import LineCollection

%matplotlib notebook
ox.config(log_console=True, use_cache=True)

# Package versions

In [6]:
print("The version of osmnx is :", ox.__version__)
print("The version of networkx is :", nx.__version__)

The version of osmnx is : 0.11.4
The version of networkx is : 2.4


# Function to display the graph according to the type of transport

In [ ]:
def type_transport(transport):
    G = ox.graph_from_place(
        'Montpellier, Hérault, France', network_type=transport)
    origin_point = ox.geo_utils.geocode('Maison du Lez, Montpellier, France')
    destination_point = ox.geo_utils.geocode(
        'Place Eugène Bataillon, Montpellier, France')
    origin_node = ox.get_nearest_node(G, origin_point)
    destination_node = ox.get_nearest_node(G, destination_point)
    route = nx.shortest_path(G, origin_node, destination_node)
    fig, ax = ox.plot_graph_route(G, route, origin_point=origin_point, destination_point=destination_point)
    plt.show
    return()

# network_func.type_transport()

In [ ]:
type_transport('walk')

# Explanation of each python command

In [ ]:
# the walking network for Montpellier, create network around point
G = ox.graph_from_place(
    'Montpellier, Hérault, France', network_type='walk')

# create network around point, define origin and destination nodes
origin_point = ox.geo_utils.geocode(
    'Maison du Lez, Montpellier, France')
destination_point = ox.geo_utils.geocode(
    'Place Eugène Bataillon, Montpellier, France')  # or define a lat-long point

# define origin/desination points then get the ID of the nodes nearest to each
origin_node = ox.get_nearest_node(G, origin_point)
destination_node = ox.get_nearest_node(G, destination_point)

# find the shortest path between origin and destination nodes
route = nx.shortest_path(G, origin_node, destination_node)

# plot the route showing origin/destination lat-long points in blue
ox.plot_graph_route(G, route, origin_point=origin_point,
                    destination_point=destination_point)

# Widget by type of transport

In [ ]:
# widget of the type of transport
menu = [('car', 'drive'), ('bike', 'bike'), ('piedestrian', 'walk')]
interact(type_transport, transport=menu)

# Function to visualize the animation according to the type of transport

In [ ]:
def animation_type_transport(transport):
    G = ox.graph_from_place(
        'Montpellier, Hérault, France', network_type=transport)
    origin_point = ox.geo_utils.geocode('Maison du Lez, Montpellier, France')
    origin_node = ox.get_nearest_node(G, origin_point)
    destination_point = ox.geo_utils.geocode(
        'Place Eugène Bataillon, Montpellier, France')
    destination_node = ox.get_nearest_node(G, destination_point)
    route = nx.shortest_path(G, origin_node, destination_node)

    fig, ax = ox.plot_graph_route(G, [origin_node])
    pic = ax.scatter(G.nodes[route[0]]['x'], G.nodes[route[0]]['y'], s=50, marker='*',
                     c='b', alpha=1, zorder=6)
    lc = LineCollection([], colors='r', linewidths=4, alpha=0.2, zorder=3)
    ax.add_collection(lc)

    def animate(i):
        pic.set_offsets([G.nodes[route[i+1]]['x'], G.nodes[route[i+1]]['y']])
        lines = ox.node_list_to_coordinate_lines(G, route[:i+1], True)
        lc.set_segments(lines)
        return pic, lc

    ani = animation.FuncAnimation(
        fig, animate, frames=200, interval=100, blit=True, repeat=False)
    plt.show()
    return(ani)

In [ ]:
animation_type_transport('bike')

# Time and memory study

In [ ]:
import time
import numpy as np


start = time.time()
graphe = type_transport('walk')
end = time.time()

print("Temps passé pour exécuter la commande: {0:.5f} s.".format(end - start))

In [27]:
import leaflet as L
bikeIcon = L.icon({
    iconUrl: 'marker_bike.png',
    iconSize:     [38, 95], # size of the icon
    iconAnchor:   [22, 94], # point of the icon which will correspond to marker's location
    popupAnchor:  [-3, -76] # point from which the popup should open relative to the iconAnchor
});

ModuleNotFoundError: No module named 'leaflet'

In [28]:
class CustomIcon(Icon):
    """
    Create a custom icon, based on an image.
    Parameters
    ----------
    icon_image :  string, file or array-like object
        The data you want to use as an icon.
        * If string, it will be written directly in the output file.
        * If file, it's content will be converted as embedded in the
        output file.
        * If array-like, it will be converted to PNG base64 string
        and embedded in the output.
    icon_size : tuple of 2 int, optional
        Size of the icon image in pixels.
    icon_anchor : tuple of 2 int, optional
        The coordinates of the "tip" of the icon
        (relative to its top left corner).
        The icon will be aligned so that this point is at the
        marker's geographical location.
    shadow_image :  string, file or array-like object, optional
        The data for the shadow image. If not specified,
        no shadow image will be created.
    shadow_size : tuple of 2 int, optional
        Size of the shadow image in pixels.
    shadow_anchor : tuple of 2 int, optional
        The coordinates of the "tip" of the shadow relative to its
        top left corner (the same as icon_anchor if not specified).
    popup_anchor : tuple of 2 int, optional
        The coordinates of the point from which popups will "open",
        relative to the icon anchor.
    """
    _template = Template(u"""
        {% macro script(this, kwargs) %}
        var {{ this.get_name() }} = L.icon({{ this.options|tojson }});
        {{ this._parent.get_name() }}.setIcon({{ this.get_name() }});
        {% endmacro %}
        """)  # noqa

    def __init__(self, icon_image, icon_size=None, icon_anchor=None,
                 shadow_image=None, shadow_size=None, shadow_anchor=None,
                 popup_anchor=None):
        super(Icon, self).__init__()
        self._name = 'CustomIcon'
        self.options = parse_options(
            icon_url=image_to_url(icon_image),
            icon_size=icon_size,
            icon_anchor=icon_anchor,
            shadow_url=shadow_image and image_to_url(shadow_image),
            shadow_size=shadow_size,
            shadow_anchor=shadow_anchor,
            popup_anchor=popup_anchor,
        )



NameError: name 'Icon' is not defined

In [ ]:
import folium

